# Cross Site Exploits 

## Cross Site Request Forgery 
This type of exploit happens on a malicious web page: when viewed by the victim, it sends a forged request to a targeted website on behalf of the victim. Since the request normally comes from a third-party web page, it is called cross-site request. <br>
When a page from a website sends an HTTP request back to the website, it is called __same-site request__. If the request is sent to a different website, it is called __cross site request__. <br>
If a website embeds an image from another website, then HTTP request used to fetch the image is a cross-site request. Online advertisers use cross-site requests to help display relevant advertisements to users, such that when the user visits the advertisements, they send HTTP request to the advertisement servers. 
![CSR](./image_files/CSR.png)

The server does not know whether the HTTP request is same-site or cross-site, this is because the browser sends all the cookies belonging to the server's domain name every time it sends HTTP request. <br>
This introduces vulnerabilities, because whereas requests from the web's same pages are trusted, those coming from other sites cannot be trusted. Therefore, it is important for a website to know whether a request is cross-site or same-site. __Cross-Site Request Forgery (CSRF)__ is the scenario where a third-party website forges requests that are exactly the same as those in same-site requests. 

### The CSRF Exploit Mechanism
The attack scenario involves three parties: 
* the attacker's malicious site
* the victim's browser 
* the target website for attacker to send HTTP request 


The conditions for each are:

* The victim has to visit the malicious site set by attacker
* The victim has to have an active session to the target site at the same time. (that is, logged in) 
* The Malicious site forges a cross-site HTTP request to target website.

#### Setup on the Ubuntu VM
The two sites: http://www.csrflabegg/com and the site http://wwww.crsflabattacker.com have already been set up on localhost, one for target and the other for attacker.

### CSRF on HTTP GET services 
HTTP GET request attach data in the URL, while POST request attach data in the body. Example of HTTP GET:
```
GET /post_form.php?foo=hello&bar=world HTTP/1.1 
Host: www.example.com
Cookie: SID=somecookies 
```
The `foo=hello&bar=world` are parameters given to the HTTP request. <br>

HTTP POST request has the format: 
```
POST /post_form.php HTTP/1.1
Host: www.example.com
Cookie: SID=someothercookies
Content-Length: 19
foo=hello&bar=world
```

#### Example of an GET CSRF attack
Consider online banking application at www.bank123.com that allows user to transfer money, via HTTP GET request such as:`http://www.bank123.com/transfer.php?to=3220&amount=500`. <br>
But before the transfer, the user needs to log in, which creates an unique identification cookie for the user. Also, the HTTP GET must be a valid request. <br>
If the attacker can get a victim to view attacker's website, they can send out a forged request from the victim's machine as a cross-site request, whereas the browser would not know it. <br> 
For GET request, the attacker can embed the following HTML in its own page: 
```
<img src="http://www.bank123.com/transfer.php?to=3220&amount=500"> 
<iframe src="http://www.bank123.com/transfer.php?to=3220&amount=500"> </iframe> 
```
The code above indicates that when the website is loaded by the browser, these tags can trigger HTTP GET requests being sent to the URLs specified in the `src` attribute.

#### Attack Procedures:
* understand how the web application works: use Firefox `LiveHTTPHeader` extension to capture the HTTP GET request. 
* example in lab is to create a new account under `Elgg` and get the parameters for 'add friend' request. 
* `__egg_ts__` and `__egg_token_` are used against the CSRF attack. 
* the attacker does not need to worry about session cookie.
* Then, the attacker need to create a malicious webpage: 
```
<html>
   <body>
       <h1> this page forges HTTP GET request </h1>
       <img src="http://www.csrfllabelgg.com/action/friends/add?friend=42" 
            alt="image" width="1" height="1" /> 
   </body>
</html> 
```
the `img` tag ensures that when browser visits the page, HTTP GET request is sent automatically. The page can be hosted locally, on `www.csrflbattacker.com`, under the `/var/www/CSRF/Attacker` folder. 
* To attract the user to visit the attacker's site, the attacker could simply send the URL to the victim, or via some social engineering. 

### CSRF on HTTP POST request 
the same forgery request in this case can be achieved using Javascript 
The POST requests are mostly done with a form and a submit button: 
```
<form action="http://www.example.com/action_post.php" method="post">
Recipient Account: <input type="text" name="to" value="3220"> <br> 
Account: <input type="text" name="amount" value="500"> <br> 
<input type="submit value="submit>
</form> 
```
The user might not click submit, so a javascript can force a submit: 

```javascript
function forge_post()
{
    var fields;
    fields += "<input type='hidden' name='to' value='3220'>";
    fields += "<input type='hidden' name='amount' value='500'>";
    
    var p = document.createElement("form");
    p.action = "http://www.example.com/action_post.php";
    p.innerHTML = fields;
    p.method = "post"; 
    document.body.appendChild(p);
    p.submit(); 
}

window.onload=function() {forge_post();}
```
The above code is embeded in HTML via `<script type="text/javascript">` and `</script>` tags <br> 
This will automatically send POST request, with values hidden to the user. 

* The same procedure is applied, where attacker captures the POST HTTP header to the site 
* The attacker takes notice of the URL and parameter sessions, and look for target's parameters 
* then the modifications are only to be made in the `fields` part of the above script. 
* Samy's Worm is done in this manner on the social network app. 

### Countermeasures
* the `refresher` field for HTTP header can identify cross-site request, but could leak private information of users.
* Chrome and Opera has the same-site cookie to identify request type. `SameSite=Strict` means that cookie will not be sent along with cross-site requests; `SamSite=Lax` means that cookie will be sent along with cross-site requests if and only if they are top-level navigations.
* Secret Tokens can be used: all the same site requests include the secret token, whereas cross-site requests do not. 
* __same origin policy__: Under the policy, a web browser permits scripts contained in a first web page to access data in a second web page, but only if both web pages have the same origin. An origin is defined as a combination of URI scheme, host name, and port number. 

Elgg uses the `__elgg_ts__` and `__elgg_token__` as countermeasures, 

## Cross Site Scripting Attack